In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input
import matplotlib.pyplot as plt
import tensorflow as tf
import random

import os 
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

In [2]:
CURRENCY = "SOL"
CLOSE = f"Close_{CURRENCY}"

seed = random.randint(0, 1000000)
seed = 122751
print(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

122751


In [3]:
dataset_pickle = '../../data/federalfinancegestion/dataset.pkl'
df = pd.read_pickle(dataset_pickle)

# Remove all string (object type) columns from the dataset
data_numeric = df.select_dtypes(exclude=['object'])

# Finding the column that closely matches "Close_BTC" in the numeric dataset
close_btc_column_numeric = [col for col in data_numeric.columns if "Close_BTC" in col]

close_btc_column_numeric = close_btc_column_numeric[0]
# Calculate the correlation of all columns with Close_BTC in the numeric dataset
correlation_with_close_btc_numeric = data_numeric.corr()[close_btc_column_numeric].sort_values(ascending=False)

# Extract the top 10 most correlated feature names
top_feature = correlation_with_close_btc_numeric.head(10).index.tolist()
include = top_feature

In [4]:
# Load data
df.fillna(0, inplace=True)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

start_date_training = pd.to_datetime('2017-08-01')
end_date_training = pd.to_datetime('2022-08-31')
start_date_testing = pd.to_datetime('2022-09-01')
end_date_testing = pd.to_datetime('2023-04-30')

In [5]:
train_df = df.loc[start_date_training:end_date_training]
test_df = df.loc[start_date_testing:end_date_testing]

In [6]:
# Calculating the logarithmic difference of 'Close'
df['Close_Log'] = np.log(df[CLOSE])  # First, compute the log of 'Close'
df['Log_Diff_Close'] = df['Close_Log'].diff().fillna(0)  # Then compute the difference and handle NaN

c:\Users\codya\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
# Include features
train = train_df[include].values
test = test_df[include].values

In [8]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

In [9]:
# # # SDAE
input_layer = Input(shape=(len(include),))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)

decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(len(include), activation='sigmoid')(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(train_scaled, train_scaled, epochs=150, batch_size=32, shuffle=True, validation_split=0.2)

# Get the encoded representation of the data
encoded_features = autoencoder.predict(train_scaled)

def create_sequences(data, sequence_length):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:(i + sequence_length), :]
        y = data[i + sequence_length, 0]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

sequence_length = 5  # Number of days used to predict the next day
X_train, y_train = create_sequences(train_scaled, sequence_length)
X_test, y_test = create_sequences(test_scaled, sequence_length)

# Reshape inputs for LSTM [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], len(include)))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], len(include)))


Epoch 1/150

33/33 [==============================] - 1s 12ms/step - loss: 0.1212 - val_loss: 0.0905
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0340 - val_loss: 0.0543
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0104 - val_loss: 0.0210
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0070 - val_loss: 0.0159
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0052 - val_loss: 0.0147
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0041 - val_loss: 0.0142
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0036 - val_loss: 0.0136
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0032 - val_loss: 0.0141
Epoch 9/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0137
Epoch 10/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0024 - val_loss: 0.0132
Epoch 

In [10]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(sequence_length, len(include))))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 50)             12200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense_6 (Dense)             (None, 25)                1275      
                                                                 
 dense_7 (Dense)             (None, 1)                 26        
                                                                 
Total params: 33701 (131.64 KB)
Trainable params: 33701 (131.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
model.fit(X_train, y_train, batch_size=64, epochs=150)

Epoch 1/150
21/21 [==============================] - 4s 5ms/step - loss: 0.0241
Epoch 2/150
21/21 [==============================] - 0s 5ms/step - loss: 0.0052
Epoch 3/150
21/21 [==============================] - 0s 5ms/step - loss: 0.0033
Epoch 4/150
21/21 [==============================] - 0s 5ms/step - loss: 0.0021
Epoch 5/150
21/21 [==============================] - 0s 5ms/step - loss: 0.0015
Epoch 6/150
21/21 [==============================] - 0s 5ms/step - loss: 0.0012
Epoch 7/150
21/21 [==============================] - 0s 6ms/step - loss: 0.0012
Epoch 8/150
21/21 [==============================] - 0s 5ms/step - loss: 0.0010
Epoch 9/150
21/21 [==============================] - 0s 5ms/step - loss: 9.7843e-04
Epoch 10/150
21/21 [==============================] - 0s 5ms/step - loss: 0.0010
Epoch 11/150
21/21 [==============================] - 0s 5ms/step - loss: 0.0010
Epoch 12/150
21/21 [==============================] - 0s 5ms/step - loss: 0.0012
Epoch 13/150
21/21 [=============

In [12]:
# Predict values
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(np.concatenate((predictions, np.zeros((predictions.shape[0], len(include)-1))), axis=1))[:, 0]

# Slice the actual prices to align with the predictions
actual_prices_aligned = df.loc[start_date_testing:end_date_testing, CLOSE][-len(predictions):]
predictions = pd.DataFrame(predictions, index=actual_prices_aligned.index, columns=['Predicted_Close'])

# Convert actual_prices_aligned to a DataFrame
actual_prices_aligned_df = pd.DataFrame(actual_prices_aligned, columns=[CLOSE])

# Calculating the logarithm of predictions and then the logarithmic difference
actual_prices_aligned_df['Predicted_Close'] = predictions
actual_prices_aligned_df['Predicted_Close'] = np.log(actual_prices_aligned_df['Predicted_Close'])
log_diff_predicted = actual_prices_aligned_df['Predicted_Close'].diff().fillna(0)
log_diff_actual = pd.DataFrame(df["Log_Diff_Close"], index=actual_prices_aligned.index, columns=['Log_Diff_Close'])

5/5 [==============================] - 1s 2ms/step


In [13]:
prediction_df = pd.concat([log_diff_actual, log_diff_predicted], axis=1)
prediction_df.rename(columns={'Log_Diff_Close': 'Actual Log Diff Close', 'Predicted Close': 'Predicted Log Diff Close'}, inplace=True)
output_path = os.path.join(os.getcwd(), '../../data/output', "prediction_"+CLOSE+".xlsx")
prediction_df.to_excel(output_path, index=False)

prediction_df

,Actual Log Diff Close,Predicted_Close
date,,
2022-09-08,0.027754,0.000000
2022-09-09,0.031051,0.000614
2022-09-12,0.076865,0.061528
2022-09-13,-0.126232,0.052149
2022-09-14,0.033376,-0.046887
...,...,...
2023-03-20,0.024241,0.063270
2023-03-21,0.017914,0.031912
2023-03-22,-0.050523,0.027677


In [14]:
money = 10000
money_buy_and_hold = 10000
money_evolution = []
money_evolution_buy_and_hold = []
backtest_values = []
for i in range(len(log_diff_predicted)):
    money_buy_and_hold = money_buy_and_hold * (1 + log_diff_actual.iloc[i][0])
    money_evolution_buy_and_hold.append(money_buy_and_hold)
    if log_diff_predicted[i] >= 0:
        money = money * (1 + log_diff_actual.iloc[i][0])
    else:
        money = money * (1 - log_diff_actual.iloc[i][0])
    money_evolution.append(money)
    cur_date = log_diff_actual.index[i].strftime("%Y-%m-%d")
    cur_rendement_predit = log_diff_predicted.iloc[i]
    cur_vol_empirique = "?"
    cur_prediction = "bull" if log_diff_predicted[i] > 0 else "bear"
    cur_alpha = "100%"
    cur_rendement_observe = log_diff_actual.iloc[i][0]
    cur_valeur_portefeuille = money
    cur_valeur_crypto = df.loc[cur_date, CLOSE]
    backtest_values.append([cur_date, cur_rendement_predit, cur_vol_empirique, cur_prediction, cur_alpha, cur_rendement_observe, cur_valeur_portefeuille, cur_valeur_crypto])
backtest = pd.DataFrame(backtest_values, columns=['date', 'rendement_predit', 'vol_empirique', 'prediction', 'alpha(expo_crypto)', 'rendement_observe', 'valeur_portefeuille', 'valeur_crypto'])
output_path = os.path.join(os.getcwd(), '../../data/output', "backtest_"+CLOSE+".xlsx")
backtest.to_excel(output_path, index=False)
backtest


,date,rendement_predit,vol_empirique,prediction,alpha(expo_crypto),rendement_observe,valeur_portefeuille,valeur_crypto
0,2022-09-08,0.000000,?,bear,100%,0.027754,10277.544210,33.61
1,2022-09-09,0.000614,?,bull,100%,0.031051,10596.673448,34.67
2,2022-09-12,0.061528,?,bull,100%,0.076865,11411.185607,37.44
3,2022-09-13,0.052149,?,bull,100%,-0.126232,9970.727797,33.00
4,2022-09-14,-0.046887,?,bear,100%,0.033376,9637.943179,34.12
...,...,...,...,...,...,...,...,...
137,2023-03-20,0.063270,?,bull,100%,0.024241,16572.850521,22.13
138,2023-03-21,0.031912,?,bull,100%,0.017914,16869.729943,22.53
139,2023-03-22,0.027677,?,bull,100%,-0.050523,16017.425816,21.42
140,2023-03-23,-0.016522,?,bear,100%,0.033964,15473.413220,22.16


In [15]:
mse = mean_squared_error(actual_prices_aligned, predictions)

# Calculate RMSE
rmse = math.sqrt(mean_squared_error(actual_prices_aligned, predictions))

# Calculate MAE
mae = mean_absolute_error(actual_prices_aligned, predictions)

# Calculate RMSE Log diff
rmse_log = math.sqrt(mean_squared_error(log_diff_actual, log_diff_predicted))

# Calculate MAE Log diff
mae_log = mean_absolute_error(log_diff_actual, log_diff_predicted)

# Convert predictions DataFrame column to a NumPy array and flatten
predictions_array = predictions.iloc[:, 0].to_numpy().flatten()

# Now calculate MAPE
# Ensure actual_prices_aligned is also a NumPy array for consistency
actual_prices_aligned_array = actual_prices_aligned.to_numpy()

mape = np.mean(np.abs((actual_prices_aligned_array - predictions_array) / actual_prices_aligned_array)) * 100

# Handle division by zero or NaN values
mape = np.nan_to_num(mape)


# Ensure actual_prices_aligned and predictions are correctly aligned and are numpy arrays
actual_values = actual_prices_aligned.values
predicted_values = predictions.values

# U-Theil1
u_theil1 = np.sqrt(np.sum(np.square(predicted_values - actual_values)) / np.sum(np.square(np.diff(actual_values))))

# U-Theil2
u_theil2 = (np.sqrt(np.mean(np.square(predicted_values - actual_values))) / np.sqrt(np.mean(np.square(actual_values))))


# Calculate the Dielbod-Mariano
# Error from your model
model_errors = actual_prices_aligned_array - predictions_array

# Error from a naive model (using previous value as prediction)
naive_errors = actual_prices_aligned_array - np.roll(actual_prices_aligned_array, 1)
naive_errors[0] = 0  # handle the first element which doesn't have a previous value
# Difference in errors
error_diff = model_errors - naive_errors

# Mean difference in errors
mean_diff = np.mean(error_diff)

# Variance of the differences
var_diff = np.var(error_diff)

# Number of observations
n = len(error_diff)

# Calculate DM test statistic
dm_stat = mean_diff / np.sqrt(var_diff / n)

absolute_rendement = money_evolution[-1] / 10000 - 1


data = {
    'Indicator Name': ['Mean Square Error','Root Mean Square Error', 'Mean Absolute Error','Root Mean Square Error (log)', 'Mean Absolute Error (log)', 'Mean Absolute Percentage Error',
                       'U-Theil1 Statistic', 'U-Theil2 Statistic', 'Diebold-Mariano Test', 'Rendement Absolue'],
    'Value': [mse, rmse, mae, rmse_log, mae_log, mape, u_theil1, u_theil2, dm_stat, absolute_rendement]
}

table = pd.DataFrame(data)
table['Value'] = table['Value'].apply(lambda x: round(x, 3))


output_path = os.path.join(os.getcwd(), '../../data/output', "performance_"+CLOSE+".xlsx")
table.to_excel(output_path, index=False)
table


,Indicator Name,Value
0,Mean Square Error,3.644811e+08
1,Root Mean Square Error,1.909139e+04
2,Mean Absolute Error,1.888664e+04
3,Root Mean Square Error (log),8.600000e-02
4,Mean Absolute Error (log),5.700000e-02
5,Mean Absolute Percentage Error,9.332521e+04
6,U-Theil1 Statistic,1.366891e+05
7,U-Theil2 Statistic,8.097280e+02
8,Diebold-Mariano Test,-8.071200e+01
9,Rendement Absolue,4.390000e-01


In [16]:
plt.figure(figsize=(16, 8))
plt.subplot(2, 1, 1)
plt.plot(money_evolution_buy_and_hold, label='Buy and hold', color='blue')
plt.plot(money_evolution, label='Applied strategy', color='red')
plt.title(f'Current Holdings ({CURRENCY})')
plt.ylabel('Money')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(log_diff_actual, label='Actual', color='blue')
plt.plot(log_diff_predicted, label='Predicted', color='red')
plt.title(f'Logarithmic Difference')
plt.xlabel('Time')
plt.ylabel('Log Difference')
plt.legend()

plt.show()

In [17]:
# import pickle

# # dump money_evolution and money_evolution_buy_and_hold
# with open(f'money/money_evolution_{CURRENCY}.pkl', 'wb') as f:
#     pickle.dump(money_evolution, f)
# with open(f'money/money_evolution_buy_and_hold_{CURRENCY}.pkl', 'wb') as f:
#     pickle.dump(money_evolution_buy_and_hold, f)

# # load mone_evoition from pickle file
# with open(f'money/money_evolution_BTC.pkl', 'rb') as f:
#     money_evolution1 = [10000] + pickle.load(f)
# with open(f'money/money_evolution_ETH.pkl', 'rb') as f:
#     money_evolution2 = [10000] + pickle.load(f)

# for i in range(len(money_evolution1)):
#     money_evolution1[i] += money_evolution2[i]

# with open(f'money/money_evolution_buy_and_hold_BTC.pkl', 'rb') as f:
#     money_evolution_buy_and_hold1 = pickle.load(f)
# with open(f'money/money_evolution_buy_and_hold_ETH.pkl', 'rb') as f:
#     money_evolution_buy_and_hold2 = pickle.load(f)

# for i in range(len(money_evolution_buy_and_hold1)):
#     money_evolution_buy_and_hold1[i] += money_evolution_buy_and_hold2[i]

# ## Compute % growth for both evolutions
# growth_buy_and_hold1 = (money_evolution_buy_and_hold1[-1] - money_evolution_buy_and_hold1[0]) / money_evolution_buy_and_hold1[0]
# growth1 = (money_evolution1[-1] - money_evolution1[0]) / money_evolution1[0]
# print(f'Buy and hold growth: {growth_buy_and_hold1*100:.2f}')
# print(f'Applied strategy growth: {growth1*100:.2f}')

# df = pd.DataFrame(money_evolution1, columns=['Investment_Value'])
# TRADING_DAYS_PER_YEAR = 365
# risk_free_rate = 0.01 / TRADING_DAYS_PER_YEAR
# df['daily_returns'] = df['Investment_Value'].pct_change(1)
# excess_returns = df['daily_returns'] - risk_free_rate
# sharpe_ratio = excess_returns.mean() / excess_returns.std()
# sharpe_ratio_annualized = sharpe_ratio * np.sqrt(TRADING_DAYS_PER_YEAR)
# print()

# df = pd.DataFrame(money_evolution_buy_and_hold1, columns=['Investment_Value'])
# TRADING_DAYS_PER_YEAR = 365
# risk_free_rate = 0.01 / TRADING_DAYS_PER_YEAR
# df['daily_returns'] = df['Investment_Value'].pct_change(1)
# excess_returns = df['daily_returns'] - risk_free_rate
# sharpe_ratio = excess_returns.mean() / excess_returns.std()
# sharpe_ratio_annualized = sharpe_ratio * np.sqrt(TRADING_DAYS_PER_YEAR)
# print(sharpe_ratio_annualized)




# plt.figure(figsize=(16, 8))
# plt.subplot(1, 1, 1)
# plt.plot(money_evolution_buy_and_hold1, label='Buy and hold', color='blue')
# plt.plot(money_evolution1, label='Applied strategy', color='red')
# plt.title(f'Two Kings Portfolio')
# plt.ylabel('Money')
# plt.xlabel('Time')
# plt.legend()